In [ ]:
import attk
import os
import numpy as np
import librosa
from IPython.display import display, Audio
import timeit
import random
from itertools import groupby
from operator import itemgetter
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
import subprocess
import unicodecsv
import urllib2

os.chdir('/home/sharedfolder/GitHub/sida/')

!mkdir /home/sharedfolder/GitHub/sida/___training_audio

os.chdir('/home/sharedfolder/GitHub/sida/___training_audio')


In [ ]:
## Loading label data CSV as a list of lists

csv_url = "https://raw.githubusercontent.com/hipstas/aapb-labels/master/Universal_Background_Model_AAPB_4K_Corpus/aapb_ubm_4K_2017.csv"

csv_string = urllib2.urlopen(csv_url)

train_table = []

csv_reader = unicodecsv.reader(csv_string)

for row in csv_reader:
        train_table.append(row)

train_table[:5]+['...']

In [ ]:
## Removing header row if present

if 'Media file basename' in train_table[0]:
    train_table = train_table[1:]

random.shuffle(train_table)

In [ ]:
%%capture
## Excerpting labeled WAV clips

training_audio_pathname = "/home/sharedfolder/audio_labeler/media/ubm_clips_final_July_2017/"
out_dir = '_classes_' + training_audio_pathname.strip('/').split('/')[-1]


for row in train_table:
    try:
        basename , start, duration, class_name, labeled_by = row  ## Assigning values in row to variables
        filename = str(basename + '.wav')
        start = float(start)
        end = float(start) + float(duration)
        out_pathname = str(os.path.join(out_dir, class_name.replace(' ','_')))
        try: 
            subprocess.call(['mkdir', '-p', out_pathname])
        except:
            pass
        attk.subclip(os.path.join(training_audio_pathname, filename), float(start), end, out_pathname) ## <- attk
    except Exception as e: 
        print(row)
        print(e)



In [ ]:

def extract_pairs(media_path,vowel_ranges):
    snd = AudioFileClip.AudioFileClip(media_path)
    file_duration = attk.duration(media_path)
    for pair in vowel_ranges:
        if int(pair[1]) >= 4:
            start, duration = pair
            start=float(start)*(512/44100.0)
            duration=float(duration)*(512/44100.0)
            if start + duration > file_duration:
                duration = file_duration - start
            basename = media_path.split('/')[-1][:-4]
            out_filename = basename+'_'+str(start)+'_'+str(duration)+'.wav'
            snd.subclip(float(start),float(start)+float(duration)).write_audiofile(os.path.join('_vowel_clips',out_filename))


def batch_extract_vowels(media_dir):

    starting_location = os.getcwd()
    
    os.chdir(media_dir)

    try: os.mkdir('_vowel_clips')
    except: pass

    filenames=[item for item in os.listdir('./') if item[-4:].lower() in ('.mp3','.wav')]

    tic=timeit.default_timer()

    for filename in filenames:
        try:
            vowel_bools = attk.get_vowel_segments(filename)
            vowel_ranges = attk.labels_to_ranges(vowel_bools, label=True)
            extract_pairs(filename,vowel_ranges)
        except: print("***** ERROR: "+filename)

    print("Time elapsed: "+str(timeit.default_timer() - tic))

    os.chdir(starting_location)


In [ ]:
os.chdir('/home/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017')

In [ ]:
for class_dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    batch_extract_vowels(class_dir_name)